In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
import math

class SimplePeopleDensityMonitor:
    def __init__(self, video_path, max_capacity=20, warning_threshold=0.7):
        self.video_path = video_path
        self.max_capacity = max_capacity
        self.warning_threshold = warning_threshold
        
        # Load YOLO model for person detection
        self.model = YOLO('yolov8n.pt')  # You can use yolov5s.pt too
        
        # Tracking variables
        self.people_in_area = {}  # Track people currently in area
        self.entry_count = 0
        self.exit_count = 0
        self.current_people = 0
        
        # Area coordinates (will be set by user)
        self.area_coords = []
        self.area_defined = False
        
    def define_area(self, frame):
        """Let user define the green monitoring area by clicking"""
        def mouse_callback(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDOWN:
                self.area_coords.append((x, y))
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
                cv2.imshow('Define Area', frame)
                
                if len(self.area_coords) >= 3:
                    # Draw the area
                    pts = np.array(self.area_coords, np.int32)
                    cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
                    cv2.imshow('Define Area', frame)
        
        cv2.imshow('Define Area', frame)
        cv2.setMouseCallback('Define Area', mouse_callback)
        
        print("Click to define area corners. Press 'q' when done.")
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q') and len(self.area_coords) >= 3:
                self.area_defined = True
                cv2.destroyWindow('Define Area')
                break
    
    def is_point_in_area(self, point):
        """Check if a point is inside the defined area"""
        if not self.area_defined:
            return False
        
        area_pts = np.array(self.area_coords, np.int32)
        return cv2.pointPolygonTest(area_pts, point, False) >= 0
    
    def calculate_area_size(self):
        """Calculate area size in pixels"""
        if len(self.area_coords) < 3:
            return 0
        
        area_pts = np.array(self.area_coords, np.int32)
        return cv2.contourArea(area_pts)
    
    def calculate_density(self):
        """Calculate people density as percentage"""
        if self.max_capacity == 0:
            return 0
        return (self.current_people / self.max_capacity) * 100
    
    def detect_people(self, frame):
        """Detect people in frame using YOLO"""
        results = self.model(frame, classes=[0], verbose=False)  # Class 0 = person
        
        detections = []
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().numpy()
                    
                    if confidence > 0.5:  # Confidence threshold
                        center_x = int((x1 + x2) / 2)
                        center_y = int((y1 + y2) / 2)
                        detections.append({
                            'center': (center_x, center_y),
                            'bbox': (int(x1), int(y1), int(x2), int(y2)),
                            'confidence': confidence
                        })
        
        return detections
    
    def track_people(self, detections, frame_id):
        """Simple tracking based on distance"""
        current_frame_people = set()
        
        for detection in detections:
            center = detection['center']
            
            # Check if person is in defined area
            if self.is_point_in_area(center):
                # Simple tracking: find closest existing person
                min_distance = float('inf')
                closest_person_id = None
                
                for person_id, person_data in self.people_in_area.items():
                    last_pos = person_data['last_position']
                    distance = math.sqrt((center[0] - last_pos[0])**2 + 
                                       (center[1] - last_pos[1])**2)
                    
                    if distance < min_distance and distance < 50:  # 50 pixel threshold
                        min_distance = distance
                        closest_person_id = person_id
                
                if closest_person_id:
                    # Update existing person
                    self.people_in_area[closest_person_id]['last_position'] = center
                    self.people_in_area[closest_person_id]['last_seen'] = frame_id
                    current_frame_people.add(closest_person_id)
                else:
                    # New person entered
                    new_id = f"person_{len(self.people_in_area)}_{frame_id}"
                    self.people_in_area[new_id] = {
                        'last_position': center,
                        'last_seen': frame_id,
                        'entry_frame': frame_id
                    }
                    self.entry_count += 1
                    current_frame_people.add(new_id)
        
        # Remove people who left the area
        people_to_remove = []
        for person_id in self.people_in_area:
            if person_id not in current_frame_people:
                if frame_id - self.people_in_area[person_id]['last_seen'] > 10:  # Lost for 10 frames
                    people_to_remove.append(person_id)
        
        for person_id in people_to_remove:
            del self.people_in_area[person_id]
            self.exit_count += 1
        
        self.current_people = len(self.people_in_area)
    
    def draw_info(self, frame):
        """Draw area, count, and warning info on frame"""
        if self.area_defined:
            # Draw defined area
            pts = np.array(self.area_coords, np.int32)
            
            # Calculate density and set color
            density = self.calculate_density()
            
            if density >= self.warning_threshold * 100:
                color = (0, 0, 255)  # Red warning
                warning_text = "WARNING: AREA OVERCROWDED!"
            elif density >= 50:
                color = (0, 165, 255)  # Orange
                warning_text = ""
            else:
                color = (0, 255, 0)  # Green
                warning_text = ""
            
            # Fill area with transparent color
            overlay = frame.copy()
            cv2.fillPoly(overlay, [pts], color)
            cv2.addWeighted(frame, 0.7, overlay, 0.3, 0, frame)
            
            # Draw border
            cv2.polylines(frame, [pts], True, color, 3)
            
            # Display information
            info_y = 30
            cv2.putText(frame, f"People in Area: {self.current_people}", 
                       (10, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.putText(frame, f"Density: {density:.1f}%", 
                       (10, info_y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.putText(frame, f"Entries: {self.entry_count} | Exits: {self.exit_count}", 
                       (10, info_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.putText(frame, f"Max Capacity: {self.max_capacity}", 
                       (10, info_y + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Warning message
            if warning_text:
                cv2.putText(frame, warning_text, 
                           (10, info_y + 130), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
        
        return frame
    
    def process_video(self):
        """Main processing function"""
        cap = cv2.VideoCapture(self.video_path)
        
        # Get first frame to define area
        ret, first_frame = cap.read()
        if not ret:
            print("Error reading video")
            return
        
        # Let user define the area
        self.define_area(first_frame.copy())
        
        frame_id = 0
        
        print("Processing video... Press 'q' to quit")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Detect people
            detections = self.detect_people(frame)
            
            # Track people in/out of area
            self.track_people(detections, frame_id)
            
            # Draw information
            frame = self.draw_info(frame)
            
            # Show frame
            cv2.imshow('People Density Monitor', frame)
            
            # Check for quit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            frame_id += 3
        
        cap.release()
        cv2.destroyAllWindows()

# Usage
if __name__ == "__main__":
    # Initialize monitor
    monitor = SimplePeopleDensityMonitor(
        video_path="testing9.mp4",  # Your video file
        max_capacity=10,              # Maximum people allowed
        warning_threshold=0.1         # 70% warning threshold
    )
    
    # Start processing
    monitor.process_video()


Click to define area corners. Press 'q' when done.
Processing video... Press 'q' to quit


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO
import math

class SimplePeopleDensityMonitor:
    def __init__(self, video_path, max_capacity=20, warning_threshold=0.7):
        self.video_path = video_path
        self.max_capacity = max_capacity
        self.warning_threshold = warning_threshold
        
        # Load YOLOv11 model for person detection
        self.model = YOLO('yolo11n.pt')  # Updated to YOLOv11
        
        # Tracking variables
        self.people_in_area = {}  # Track people currently in area
        self.entry_count = 0
        self.exit_count = 0
        self.current_people = 0
        
        # Area coordinates (will be set by user)
        self.area_coords = []
        self.area_defined = False
        
        # Person states for colored boxes
        self.person_states = {}  # Track if person is entering, exiting, or still
        
    def define_area(self, frame):
        """Let user define the green monitoring area by clicking"""
        def mouse_callback(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDOWN:
                self.area_coords.append((x, y))
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
                cv2.imshow('Define Area', frame)
                
                if len(self.area_coords) >= 3:
                    # Draw the area
                    pts = np.array(self.area_coords, np.int32)
                    cv2.polylines(frame, [pts], True, (0, 255, 0), 2)
                    cv2.imshow('Define Area', frame)
        
        cv2.imshow('Define Area', frame)
        cv2.setMouseCallback('Define Area', mouse_callback)
        
        print("Click to define area corners. Press 'q' when done.")
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q') and len(self.area_coords) >= 3:
                self.area_defined = True
                cv2.destroyWindow('Define Area')
                break
    
    def is_point_in_area(self, point):
        """Check if a point is inside the defined area"""
        if not self.area_defined:
            return False
        
        area_pts = np.array(self.area_coords, np.int32)
        return cv2.pointPolygonTest(area_pts, point, False) >= 0
    
    def calculate_area_size(self):
        """Calculate area size in pixels"""
        if len(self.area_coords) < 3:
            return 0
        
        area_pts = np.array(self.area_coords, np.int32)
        return cv2.contourArea(area_pts)
    
    def calculate_density(self):
        """Calculate people density as percentage"""
        if self.max_capacity == 0:
            return 0
        return (self.current_people / self.max_capacity) * 100
    
    def detect_people(self, frame):
        """Detect people in frame using YOLOv11"""
        results = self.model(frame, classes=[0], verbose=False)  # Class 0 = person
        
        detections = []
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().numpy()
                    
                    if confidence > 0.5:  # Confidence threshold
                        center_x = int((x1 + x2) / 2)
                        center_y = int((y1 + y2) / 2)
                        detections.append({
                            'center': (center_x, center_y),
                            'bbox': (int(x1), int(y1), int(x2), int(y2)),
                            'confidence': confidence
                        })
        
        return detections
    
    def track_people(self, detections, frame_id):
        """Enhanced tracking with person states"""
        current_frame_people = set()
        new_states = {}
        
        for detection in detections:
            center = detection['center']
            bbox = detection['bbox']
            
            # Check if person is in defined area
            if self.is_point_in_area(center):
                # Simple tracking: find closest existing person
                min_distance = float('inf')
                closest_person_id = None
                
                for person_id, person_data in self.people_in_area.items():
                    last_pos = person_data['last_position']
                    distance = math.sqrt((center[0] - last_pos[0])**2 + 
                                       (center[1] - last_pos[1])**2)
                    
                    if distance < min_distance and distance < 50:  # 50 pixel threshold
                        min_distance = distance
                        closest_person_id = person_id
                
                if closest_person_id:
                    # Update existing person - mark as STILL
                    self.people_in_area[closest_person_id]['last_position'] = center
                    self.people_in_area[closest_person_id]['last_seen'] = frame_id
                    self.people_in_area[closest_person_id]['bbox'] = bbox
                    current_frame_people.add(closest_person_id)
                    new_states[closest_person_id] = 'still'
                else:
                    # New person entered - mark as ENTERING
                    new_id = f"person_{len(self.people_in_area)}_{frame_id}"
                    self.people_in_area[new_id] = {
                        'last_position': center,
                        'last_seen': frame_id,
                        'entry_frame': frame_id,
                        'bbox': bbox
                    }
                    self.entry_count += 1
                    current_frame_people.add(new_id)
                    new_states[new_id] = 'entering'
        
        # Mark people who are leaving as EXITING
        people_to_remove = []
        for person_id in self.people_in_area:
            if person_id not in current_frame_people:
                if frame_id - self.people_in_area[person_id]['last_seen'] < 5:  # Recently left
                    new_states[person_id] = 'exiting'
                elif frame_id - self.people_in_area[person_id]['last_seen'] > 10:  # Lost for 10 frames
                    people_to_remove.append(person_id)
        
        for person_id in people_to_remove:
            if person_id in self.person_states:
                del self.person_states[person_id]
            del self.people_in_area[person_id]
            self.exit_count += 1
        
        # Update person states
        self.person_states = new_states
        self.current_people = len(self.people_in_area)
    
    def draw_person_boxes(self, frame):
        """Draw colored bounding boxes for each person"""
        for person_id, person_data in self.people_in_area.items():
            bbox = person_data.get('bbox')
            if bbox is None:
                continue
                
            x1, y1, x2, y2 = bbox
            state = self.person_states.get(person_id, 'still')
            
            # Set color based on state
            if state == 'entering':
                color = (0, 255, 0)  # Green for entering
                label = "ENTRY"
            elif state == 'exiting':
                color = (0, 0, 255)  # Red for exiting
                label = "EXIT"
            else:  # still
                color = (128, 128, 128)  # Gray for still
                label = "STILL"
            
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Draw label
            label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), 
                         (x1 + label_size[0], y1), color, -1)
            cv2.putText(frame, label, (x1, y1 - 5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    def draw_info(self, frame):
        """Draw area, count, and warning info on frame"""
        if self.area_defined:
            # Draw defined area
            pts = np.array(self.area_coords, np.int32)
            
            # Calculate density and set area color
            density = self.calculate_density()
            
            # Check if current people exceed max capacity
            if self.current_people > self.max_capacity:
                area_color = (0, 0, 255)  # Red warning - EXCEEDED CAPACITY
                warning_text = f"WARNING: CAPACITY EXCEEDED! ({self.current_people}/{self.max_capacity})"
            elif density >= self.warning_threshold * 100:
                area_color = (0, 165, 255)  # Orange warning - APPROACHING CAPACITY
                warning_text = f"WARNING: APPROACHING CAPACITY! ({self.current_people}/{self.max_capacity})"
            else:
                area_color = (0, 255, 0)  # Green - NORMAL
                warning_text = ""
            
            # Fill area with transparent color
            overlay = frame.copy()
            cv2.fillPoly(overlay, [pts], area_color)
            cv2.addWeighted(frame, 0.7, overlay, 0.3, 0, frame)
            
            # Draw border
            cv2.polylines(frame, [pts], True, area_color, 3)
            
            # Draw person bounding boxes
            self.draw_person_boxes(frame)
            
            # Display information
            info_y = 30
            cv2.putText(frame, f"People in Area: {self.current_people}/{self.max_capacity}", 
                       (10, info_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.putText(frame, f"Density: {density:.1f}%", 
                       (10, info_y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.putText(frame, f"Entries: {self.entry_count} | Exits: {self.exit_count}", 
                       (10, info_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Status indicator
            status = "EXCEEDED" if self.current_people > self.max_capacity else "NORMAL"
            status_color = (0, 0, 255) if self.current_people > self.max_capacity else (0, 255, 0)
            cv2.putText(frame, f"Status: {status}", 
                       (10, info_y + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, status_color, 2)
            
            # Warning message
            if warning_text:
                # Background for warning text
                text_size = cv2.getTextSize(warning_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0]
                cv2.rectangle(frame, (5, info_y + 110), 
                             (text_size[0] + 15, info_y + 150), (0, 0, 0), -1)
                cv2.putText(frame, warning_text, 
                           (10, info_y + 135), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
            # Legend for box colors
            legend_y = frame.shape[0] - 100
            cv2.putText(frame, "Legend:", (10, legend_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.rectangle(frame, (10, legend_y + 10), (30, legend_y + 30), (0, 255, 0), -1)  # Green
            cv2.putText(frame, "Entry", (35, legend_y + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.rectangle(frame, (10, legend_y + 35), (30, legend_y + 55), (0, 0, 255), -1)  # Red
            cv2.putText(frame, "Exit", (35, legend_y + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.rectangle(frame, (10, legend_y + 60), (30, legend_y + 80), (128, 128, 128), -1)  # Gray
            cv2.putText(frame, "Still", (35, legend_y + 75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame
    
    def process_video(self):
        """Main processing function"""
        cap = cv2.VideoCapture(self.video_path)
        
        # Get first frame to define area
        ret, first_frame = cap.read()
        if not ret:
            print("Error reading video")
            return
        
        # Let user define the area
        self.define_area(first_frame.copy())
        
        frame_id = 0
        
        print("Processing video... Press 'q' to quit")
        print("Box Colors: GREEN=Entry, RED=Exit, GRAY=Still")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Detect people
            detections = self.detect_people(frame)
            
            # Track people in/out of area
            self.track_people(detections, frame_id)
            
            # Draw information and boxes
            frame = self.draw_info(frame)
            
            # Show frame
            cv2.imshow('People Density Monitor - YOLOv11', frame)
            
            # Check for quit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            frame_id += 1
        
        cap.release()
        cv2.destroyAllWindows()

# Usage
if __name__ == "__main__":
    # Initialize monitor
    monitor = SimplePeopleDensityMonitor(
        video_path="testing9.mp4",  # Your video file
        max_capacity=5,              # Maximum people allowed
        warning_threshold=0.6         # 70% warning threshold
    )
    
    # Start processing
    monitor.process_video()


Click to define area corners. Press 'q' when done.
Processing video... Press 'q' to quit
Box Colors: GREEN=Entry, RED=Exit, GRAY=Still
